In [ ]:
# !pip install uv
# !uv pip install git+https://github.com/ysenarath/AIT-724-HandsOn-2.git

In [ ]:
import copy
import gzip
import os
import random
import shutil
import zipfile
from datetime import timedelta
from pathlib import Path
from typing import Any

import community as community_louvain
import holoviews as hv
import hvplot.networkx as hvnx
import igraph as ig
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from networkx.drawing.layout import forceatlas2_layout as _forceatlas2_layout
from tqdm import auto as tqdm

hv.extension("bokeh")


def forceatlas2_layout(*args, **kwargs) -> dict:
    return _forceatlas2_layout(*args, **kwargs)

# Download the Files

- The files are available at https://snap.stanford.edu/data/higgs-twitter.html

- Base URL: https://snap.stanford.edu/data/higgs-*

| File Name                   | Description                                                                                    |
| --------------------------- | ---------------------------------------------------------------------------------------------- |
| social_network.edgelist.gz  | Friends/follower graph (directed)                                                              |
| retweet_network.edgelist.gz | Graph of who retweets whom (directed and weighted)                                             |
| reply_network.edgelist.gz   | Graph of who replies to who (directed and weighted)                                            |
| mention_network.edgelist.gz | Graph of who mentions whom (directed and weighted)                                             |
| higgs-activity_time.txt.gz  | The dataset provides information about activity on Twitter during the discovery of Higgs boson |


In [ ]:
def download_file(url, dest_folder):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    local_filename = os.path.join(dest_folder, url.split("/")[-1])
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        total_size = int(r.headers.get("content-length", 0))
        with (
            open(local_filename, "wb") as f,
            tqdm.tqdm(
                desc=local_filename,
                total=total_size,
                unit="iB",
                unit_scale=True,
                unit_divisor=1024,
            ) as bar,
        ):
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
                bar.update(len(chunk))
    return local_filename


def unzip_file(zip_path, extract_to):
    if zipfile.is_zipfile(zip_path):
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(extract_to)
    elif zip_path.endswith(".gz"):
        with gzip.open(zip_path, "rb") as f_in:
            with open(os.path.splitext(zip_path)[0], "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)


def main():
    base_url = "https://snap.stanford.edu/data/higgs-"
    files = [
        "social_network.edgelist.gz",
        "retweet_network.edgelist.gz",
        "reply_network.edgelist.gz",
        "mention_network.edgelist.gz",
        "activity_time.txt.gz",
    ]
    dest_folder = "higgs_twitter_data"

    for file_name in files:
        url = base_url + file_name
        output_path = Path(dest_folder) / ("higgs-" + file_name.replace(".gz", ""))
        if output_path.exists():
            print(f"{file_name} already exists. Skipping download.")
            continue
        print(f"Downloading {url}...")
        downloaded_file = download_file(url, dest_folder)
        print(f"Unzipping {downloaded_file}...")
        unzip_file(downloaded_file, dest_folder)
        os.remove(downloaded_file)  # Remove the compressed file after extraction
        print(f"Finished processing {file_name}.\n")


main()

# Understanding Edge List File Format

- For this exercise, we will only explore the social network (followers/friends) edge list file: `social_network.edgelist.gz` and the activity time file: `higgs-activity_time.txt.gz`
- First we extract the largest connected component from the social network and use this subgraph for our analysis.
- This file contains four columns: userA userB timestamp interaction type
- every edge is directed from userA to userB
- First let's try to load this file using pandas and explore its contents.


In [ ]:
main_event_date = pd.to_datetime("2012-07-04")

In [ ]:
# Let's create a network graph using NetworkX
follower_df = pd.read_csv(
    os.path.join("higgs_twitter_data", "higgs-social_network.edgelist"),
    sep=" ",
    header=None,
    names=["source", "target"],
).reset_index(drop=True)

# Create a directed graph from the follower data
F: nx.DiGraph = nx.from_pandas_edgelist(
    follower_df,
    source="source",
    target="target",
    create_using=nx.DiGraph,
)

# Let's identify the largest weakly connected component (LCC) for the follower graph
largest_wcc = max(nx.weakly_connected_components(F), key=len)

F_wcc = F.subgraph(largest_wcc)

print(
    f"Follower graph LCC has {F_wcc.number_of_nodes()} nodes and {F_wcc.number_of_edges()} edges."
)

In [ ]:
# Display the first 5 nodes in the LCC
[n for i, n in enumerate(F_wcc.nodes()) if i < 5]

In [ ]:
df = pd.read_csv(
    os.path.join("higgs_twitter_data", "higgs-activity_time.txt"),
    sep=" ",
    header=None,
    names=["source", "target", "timestamp", "type"],
)
print(f"Total interactions in dataset: {len(df)}")

# filter interactions where both source and target are in the follower LCC
df = df[
    (df["source"].isin(largest_wcc)) & (df["target"].isin(largest_wcc))
].reset_index(drop=True)

print(f"Total interactions after filtering to follower LCC: {len(df)}")

# Convert timestamp to datetime
df["datetime"] = pd.to_datetime(df["timestamp"], unit="s")

# let's look at the distribution of interaction types
interaction_counts = df["type"].value_counts()
print("Interaction type distribution:")
print(interaction_counts)

# Get {DT} days before and after the main event date
# DT = 1
# start_date = main_event_date - pd.Timedelta(days=DT)
# end_date = main_event_date + pd.Timedelta(days=DT)
# df = df[(df["datetime"] >= start_date) & (df["datetime"] <= end_date)]
df = df[df["type"] == "RE"]
start_date = df["datetime"].min()

# If one is interested in building a network of how information flows, then the direction of RT should be reversed when used in the analysis.
#   ref. https://snap.stanford.edu/data/higgs-twitter.html
df = df.rename(columns={"source": "target", "target": "source"})

total_number_of_nodes = df[["source", "target"]].stack().nunique()
total_number_of_edges = len(df)

print(f"Total number of unique nodes: {total_number_of_nodes}")
print(f"Total number of edges: {total_number_of_edges}")

df.head(5)

In [ ]:
# let's look at the time range of the interactions
min_time = df["datetime"].min()
max_time = df["datetime"].max()
print(f"Time range of interactions: {min_time} to {max_time}")

In [ ]:
G: nx.DiGraph = nx.from_pandas_edgelist(
    df,
    source="source",
    target="target",
    edge_attr=["datetime", "type"],
    create_using=nx.DiGraph,
)

print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

In [ ]:
# Let's show the graph with matplotlib for the first hour of data

sampled_df = df[(start_date <= df["datetime"]) & (df["datetime"] < main_event_date)]

H: nx.DiGraph = nx.from_pandas_edgelist(
    sampled_df,
    source="source",
    target="target",
    edge_attr=["datetime", "type"],
    create_using=nx.DiGraph,
)

plt.figure(figsize=(10, 10))

pos = forceatlas2_layout(
    H,
    max_iter=20,  # default is 100, higher values give more precise layout
    jitter_tolerance=0.5,  # default is 1.0, lower is more precise
    scaling_ratio=3.0,  # default is 2.0, higher values spread out the layout
    gravity=0.5,  # default is 1.0, higher values pull nodes towards the center
)

nx.draw(H, pos, with_labels=True, node_size=50, font_size=12)

plt.title("Higgs Twitter Network - First Hour of Data")

plt.show()

In [ ]:
# Visualise early diffusion: e.g., plot number of new retweeters over time.
plt.figure(figsize=(12, 6))

sns.histplot(
    df,
    x="datetime",
    bins=50,
    kde=False,
    hue="type",
    multiple="stack",
)

plt.show()


In [ ]:
# Compute degree
print("Computing node degrees...")
degree_dict = dict(G.degree())
nx.set_node_attributes(G, degree_dict, "degree")

# in degree
in_degree_dict = dict(G.in_degree())
nx.set_node_attributes(G, in_degree_dict, "in_degree")

In [ ]:
# Compute betweenness centrality

# # print("Computing betweenness centrality...")
# # takes around 15 mins-45mins
# centrality_dict = nx.betweenness_centrality(G)
# nx.set_node_attributes(G, centrality_dict, "betweenness_centrality")

# follwing is a more efficient way using igraph, commented out for now

# compute betweenness centrality for all vertices using igraph
print("Computing betweenness centrality using igraph...")
G_ig = ig.Graph.from_networkx(G)
vertex_betweenness = G_ig.betweenness()

# put it back to networkx graph
# betweenness_dict = {v.index: float(vertex_betweenness[v.index]) for v in G_ig.vs}
names = [v["_nx_name"] for v in G_ig.vs]
betweenness_dict = dict(zip(names, vertex_betweenness))
nx.set_node_attributes(G, betweenness_dict, "betweenness_centrality")

In [ ]:
# closeness centrality
print("Computing closeness centrality...")

closeness_dict = nx.closeness_centrality(G)

nx.set_node_attributes(G, closeness_dict, "closeness_centrality")

In [ ]:
# Run PageRank algorithm
print("Computing PageRank...")

pagerank_dict = nx.pagerank(G, alpha=0.85)

nx.set_node_attributes(G, pagerank_dict, "pagerank")

In [ ]:
# Compute clustering coefficient
print("Computing clustering coefficient...")

clustering_dict = nx.clustering(G.to_undirected())

nx.set_node_attributes(G, clustering_dict, "clustering_coefficient")

In [ ]:
# Add modularity class using the Louvain method
partition = community_louvain.best_partition(G.to_undirected())

nx.set_node_attributes(G, partition, "modularity_class")

In [ ]:
# Common measure for quantifying influence of bloggers is to use in-degree centrality
# - This is the number of users who follow a person on Twitter.
# In-links are sparse
# - More detailed analysis is required to measure influence

in_degrees: dict[str, float] = nx.get_node_attributes(G, "in_degree")
threshold = np.percentile(list(in_degrees.values()), 99)  # top 1%

influencers = [n for n, deg in in_degrees.items() if deg >= threshold]
regulars = [n for n in G.nodes if n not in influencers]

print(f"Influencers: {len(influencers)} | Regulars: {len(regulars)}")

# add this back to the network G
influencer_dict = {n: n in influencers for n in G.nodes}
nx.set_node_attributes(G, influencer_dict, "is_influencer")

In [ ]:
# show node & edge attributes
print("Node attribute names for first node:")
first_node = list(G.nodes)[0]
print(G.nodes[first_node].keys())
print("Edges attribute names for first node:")
first_edge = list(G.edges)[0]
print(G.edges[first_edge].keys())

In [ ]:
def group_timestamps(
    G: nx.DiGraph,
    node: Any,
    time_window: timedelta | None,
    timestamp_col: str = "datetime",
) -> list[list[pd.Timestamp]]:
    items = []
    for source, target, data in G.edges(node, data=True):
        dt = data[timestamp_col]
        items.append(dt)
    items = sorted(items)
    if time_window is None:
        # all in one cluster
        clusters = [[items]]
    else:
        clusters = [[x] for x in items]
        i = 0
        while i < len(clusters) - 1:
            if (clusters[i + 1][0] - clusters[i][0]) < time_window:
                clusters[i].extend(clusters[i + 1])
                del clusters[i + 1]
            else:
                i += 1
    return clusters


def build_cascade_from_root(
    G: nx.DiGraph,
    root: Any,
    timestamp_cluster: list[pd.Timestamp],
    visited: set,
    time_window: timedelta | None = None,
    timestamp_col: str = "datetime",
) -> nx.DiGraph:
    # Start a cascade
    cascade_nodes = set([root])
    cascade_edges = dict()
    queue = [(root, ts) for ts in timestamp_cluster]
    visited.add(root)
    while queue:
        current, t_curr = queue.pop(0)
        neighbors = sorted(
            G.neighbors(current),
            key=lambda nbr: G.get_edge_data(current, nbr, default={})[timestamp_col],
        )
        for nbr in neighbors:
            edge = G.get_edge_data(current, nbr, default={})
            # typeof datetime is pandas.Timestamp
            t_edge = edge[timestamp_col]
            # convert to real time and print
            if time_window is not None and (t_edge - t_curr > time_window):
                continue
            if nbr not in visited:
                cascade_nodes.add(nbr)
                cascade_edges[(current, nbr)] = edge
                queue.append((nbr, t_edge))
                visited.add(nbr)
    cascade = nx.DiGraph()
    cascade.add_nodes_from(cascade_nodes)
    cascade_edges_with_data = [
        (src, tgt, data) for (src, tgt), data in cascade_edges.items()
    ]
    cascade.add_edges_from(cascade_edges_with_data)
    return cascade


def build_cascades(
    G: nx.DiGraph,
    time_window: timedelta | None = None,
    min_cascade_size: int = 1,
    timestamp_col: str = "datetime",
) -> list[nx.DiGraph]:
    cascades = []
    visited = set()
    for root in G.nodes():
        if root in visited:
            continue
        for timestamp_cluster in group_timestamps(
            G, root, time_window=time_window, timestamp_col=timestamp_col
        ):
            cascade = build_cascade_from_root(
                G,
                root,
                timestamp_cluster=timestamp_cluster,
                visited=visited,
                time_window=time_window,
                timestamp_col=timestamp_col,
            )
            cascades.append(cascade)
    # filter cascades with size > 1
    cascades = [c for c in cascades if len(c) > min_cascade_size]
    return cascades

In [ ]:
cascades = build_cascades(G, time_window=timedelta(minutes=1), min_cascade_size=10)
print(f"Total cascades found: {len(cascades)}")
largest_cascade = max(cascades, key=len)
print(f"Largest cascade size: {len(largest_cascade)}")
# size of the cascades
cascade_sizes = [len(c) for c in cascades]
plt.figure(figsize=(10, 6))
sns.histplot(cascade_sizes, bins=30, kde=False)
plt.title("Cascade Size Distribution")
plt.xlabel("Cascade Size")
plt.ylabel("Frequency")
plt.show()

In [ ]:
cascades = build_cascades(G, time_window=None, min_cascade_size=10)
print(f"Total cascades found: {len(cascades)}")
largest_cascade = max(cascades, key=len)
print(f"Largest cascade size: {len(largest_cascade)}")
# size of the cascades
cascade_sizes = [len(c) for c in cascades]
plt.figure(figsize=(10, 6))
sns.histplot(cascade_sizes, bins=30, kde=False)
plt.title("Cascade Size Distribution")
plt.xlabel("Cascade Size")
plt.ylabel("Frequency")
plt.show()

In [ ]:
def show_network_hvplot(
    G: nx.Graph, sample_size: int | None = None, date_format: str = "%Y-%m-%d"
):
    # Sample graph to manageable size
    if sample_size is not None and len(G.nodes) > sample_size:
        nodes_sample = random.sample(list(G.nodes()), sample_size)
        G_copy = copy.deepcopy(G.subgraph(nodes_sample))
    else:
        G_copy = copy.deepcopy(G)

    # Convert Timestamp attributes to strings
    for n, data in G_copy.nodes(data=True):
        for k, v in list(data.items()):
            if isinstance(v, pd.Timestamp):
                data[k] = v.strftime(date_format)

    for u, v, data in G_copy.edges(data=True):
        for k, val in list(data.items()):
            if isinstance(val, pd.Timestamp):
                data[k] = val.strftime(date_format)

    # Compute layout (spring layout works well for most graphs)
    pos = forceatlas2_layout(
        G_copy,
        max_iter=800,
        jitter_tolerance=1.0,
        scaling_ratio=2.0,
        gravity=1.0,
    )

    # Create hvplot
    plot = hvnx.draw(
        G_copy,
        pos,
        node_size=100,
        node_color="lightblue",
        edge_color="gray",
        with_labels=True,
    ).opts(
        width=900,
        height=700,
        tools=["hover", "box_zoom", "wheel_zoom", "save", "reset"],
        title=f"Interactive Graph (|N|={len(G_copy)}, |E|={len(G_copy.edges)})",
    )

    # Add hover info dynamically
    hover_data = []
    for n, d in G_copy.nodes(data=True):
        hover_data.append({"id": n, **d})
        # You can attach hover info via node attributes if needed

    return plot


show_network_hvplot(cascades[0])